## Exercise 04. A/B TESTING


1. Create a connection to the database using the `sqlite3` library.
2. Using only one query for each group, create two dataframes: `test_results` and `control_results`, with the columns "time" and "avg_diff", and only two rows.
   - "time" should have the values after and before.
   - "avg_diff" should contain the average delta for all users for the time period before and after their first visit to the page.
   - Only take into account users with observations before and after.
3. We are still not using the lab `project1`.
4. Close the connection.
5. Have the answer: Did the hypothesis turn out to be true, and does the page affect students' behavior?

In [ ]:
import pandas as pd
import sqlite3 

con=sqlite3.connect('data/checking-logs.sqlite')

query_test = """
WITH dif AS (
    SELECT 
        t.uid,
        CASE
            WHEN JULIANDAY(t.first_commit_ts) > JULIANDAY(t.first_view_ts) THEN 'after'
            ELSE 'before'
        END AS time,
        AVG((strftime('%s', t.first_commit_ts) - strftime('%s', d.deadlines, 'unixepoch')) / 3600) AS avg_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    GROUP BY t.uid, time
)
SELECT time, AVG(avg_diff) AS 'avg(diff)'
FROM dif
WHERE uid IN (
    SELECT uid
    FROM dif
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
GROUP BY time
"""
test_results=pd.read_sql(query_test, con)
test_results

,time,avg(diff)
0,after,-99.523810
1,before,-66.047619


In [ ]:
import pandas as pd
import sqlite3 

con=sqlite3.connect('../data/checking-logs.sqlite')

query_control = """
WITH dif AS (
    SELECT 
        c.uid,
        CASE
            WHEN JULIANDAY(c.first_commit_ts) > JULIANDAY(c.first_view_ts) THEN 'after'
            ELSE 'before'
        END AS time,
        AVG((strftime('%s', c.first_commit_ts) - strftime('%s', d.deadlines, 'unixepoch')) / 3600) AS avg_diff
    FROM control c
    JOIN deadlines d ON c.labname = d.labs
    WHERE c.labname != 'project1'
    GROUP BY c.uid, time
)
SELECT time, AVG(avg_diff) AS 'avg(diff)'
FROM dif
WHERE uid IN (
    SELECT uid
    FROM dif
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
GROUP BY time
"""
control_results=pd.read_sql(query_control, con)
control_results

,time,avg(diff)
0,after,-99.322222
1,before,-98.033333


In [10]:
con.close()

answer=YES